## Parte 1: Desarrollo de redes neuronales
Objetivos: 
1. Identificar columnas numéricas y columnas categóricas.
2. Diseñar una red de neuronas para resolver un problema de **regresión**.
3. Utilizar métricas para evaluar el modelo generado.

### 1.1 Datos ganaderos de Catalunya
Se trabajará únicamente con las columnas: [CODI POSTAL EXPLOTACIO,SERVEI TERRITORIAL  EXPLOTACIÓ,PROVINCIA EXPLOTACIÓ,COMARCA EXPLOTACIÓ,MUNICIPI EXPLOTACIÓ,TIPUS EXPLOTACIÓ,ESPÈCIE, INTEGRADORA, NOM ADS,CLASSIFICACIÓ ZOOTÈCNICA,DATA CLASSIFICACIÓ ZOOTÈCNICA,SISTEMA PRODUCTIU,CRITERI DE SOSTENIBILITAT,CAPACITAT PRODUCTIVA,TOTAL CAP PONEDORES,TOTAL URM,TOTAL NITROGEN].

El modelo a generar debe trabajar con TOTAL_URM o TOTAL_NITROGEN como variable a predecir. A elección del grupo. 

In [1]:
import pandas as pd 

data = pd.read_csv('carga_ganadera.csv')

columnas = [
    'CODI POSTAL EXPLOTACIO', 'SERVEI TERRITORIAL  EXPLOTACIÓ', 'PROVINCIA EXPLOTACIÓ',
    'COMARCA EXPLOTACIÓ', 'MUNICIPI EXPLOTACIÓ', 'TIPUS EXPLOTACIÓ', 'ESPÈCIE', 
    'INTEGRADORA', 'NOM ADS', 'CLASSIFICACIÓ ZOOTÈCNICA', 'DATA CLASSIFICACIÓ ZOOTÈCNICA',
    'SISTEMA PRODUCTIU', 'CRITERI DE SOSTENIBILITAT', 'CAPACITAT PRODUCTIVA', 
    'TOTAL CAP PONEDORES', 'TOTAL URM', 'TOTAL NITROGEN'
]

data = data[columnas]

In [ ]:
data.info()

In [ ]:
categorical_columns = data.select_dtypes(include=['object']).columns
categorical_columns

**1.1.1 Conversión a One-Hot Encoding**
___

In [ ]:
# 1.1.1 Convertir las columnas categóricas a one-hot encoding. (1 ptos)

from sklearn.preprocessing import OneHotEncoder

x_categorical = data[categorical_columns]
encoder = OneHotEncoder(handle_unknown='ignore')

x_encoded = encoder.fit_transform(x_categorical)

categorical_encoded_columns = [
    f'{col}_{cat}' for i, col in enumerate(x_categorical.columns) for cat in encoder.categories_[i]
]

x_encoded_df = pd.DataFrame(x_encoded.toarray(), columns=categorical_encoded_columns)
df = pd.concat([data.drop(columns=categorical_columns), x_encoded_df], axis=1)

df.head()

In [ ]:
df.shape

**1.1.2 Red Neuronal para TOTAL_URM  -  random_state=42**
___

Usamos esta sentencia de código para ver que columnas tenian NaN, lo cual no dejaba ejecutar la función de activación de sigmoid

``nan_columns = x_train.columns[x_train.isnull().any()]``  
``print('Columnas con NaN en x_train:', nan_columns)``

In [ ]:
# 1.1.2 Construir una red de neuronas para identificar el `TOTAL_URM` o `TOTAL_NITROGEN`. (3.5 ptos) 
# Se evaluará el loss function utilizado en cada capa.

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import numpy as np

tf.random.set_seed(
    42
)

target_variable = 'TOTAL URM'

x = df.drop(columns=[target_variable])
y = df[target_variable]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print('NaN en x_train:', x_train.isnull().any().any())
print('NaN en x_test:', x_test.isnull().any().any())

# Manejar NaN - Imputación con la media
imputer = SimpleImputer(strategy='mean')
imputer.fit(x_train)

x_train_imputed = pd.DataFrame(imputer.transform(x_train), columns=x_train.columns, index=x_train.index)
x_test_imputed = pd.DataFrame(imputer.transform(x_test), columns=x_test.columns, index=x_test.index)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_imputed)
x_test_scaled = scaler.transform(x_test_imputed)

print('NaN en x_train:', x_train_imputed.isnull().any().any())
print('NaN en x_test:', x_test_imputed.isnull().any().any())

model_relu = keras.models.Sequential()
model_relu.add(keras.layers.InputLayer(shape=[x_train_scaled.shape[1]])) 
model_relu.add(keras.layers.Dense(200, activation="relu"))
model_relu.add(keras.layers.Dense(100, activation="relu"))
model_relu.add(keras.layers.Dense(50, activation="relu"))
model_relu.add(keras.layers.Dense(20, activation="relu"))
model_relu.add(keras.layers.Dense(1, activation="linear"))

model_sigmoid = keras.models.Sequential()
model_sigmoid.add(keras.layers.InputLayer(shape=[x_train_scaled.shape[1]])) 
model_sigmoid.add(keras.layers.Dense(200, activation="sigmoid"))
model_sigmoid.add(keras.layers.Dense(100, activation="sigmoid"))
model_sigmoid.add(keras.layers.Dense(50, activation="sigmoid"))
model_sigmoid.add(keras.layers.Dense(20, activation="sigmoid"))
model_sigmoid.add(keras.layers.Dense(1, activation="linear"))

model_tanh = keras.models.Sequential()
model_tanh.add(keras.layers.InputLayer(shape=[x_train_scaled.shape[1]])) 
model_tanh.add(keras.layers.Dense(200, activation="tanh"))
model_tanh.add(keras.layers.Dense(100, activation="tanh"))
model_tanh.add(keras.layers.Dense(50, activation="tanh"))
model_tanh.add(keras.layers.Dense(20, activation="tanh"))
model_tanh.add(keras.layers.Dense(1, activation="linear"))

model_elu = keras.models.Sequential()
model_elu.add(keras.layers.InputLayer(shape=[x_train_scaled.shape[1]])) 
model_elu.add(keras.layers.Dense(200, activation="elu"))
model_elu.add(keras.layers.Dense(100, activation="elu"))
model_elu.add(keras.layers.Dense(50, activation="elu"))
model_elu.add(keras.layers.Dense(20, activation="elu"))
model_elu.add(keras.layers.Dense(1, activation="linear"))

**1.1.3 Evaluación de desenvolvimiento del modelo - 50 épocas**
___

In [39]:
# Compilamos ambos modelos con métricas

model_relu.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

model_sigmoid.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

model_tanh.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

model_elu.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

In [ ]:
history_relu = model_relu.fit(
    x_train_scaled, y_train,
    validation_data=(x_test_scaled, y_test),
    epochs=50,
    batch_size=32,
    verbose=1
)

history_sigmoid = model_sigmoid.fit(
    x_train_scaled, y_train,
    validation_data=(x_test_scaled, y_test),
    epochs=50,
    batch_size=32,
    verbose=1
)

history_tanh = model_tanh.fit(
    x_train_scaled, y_train,
    validation_data=(x_test_scaled, y_test),
    epochs=50,
    batch_size=32,
    verbose=1
)

history_elu = model_elu.fit(
    x_train_scaled, y_train,
    validation_data=(x_test_scaled, y_test),
    epochs=50,
    batch_size=32,
    verbose=1
)

In [ ]:
model_relu.summary()

In [ ]:
model_sigmoid.summary()

In [ ]:
model_tanh.summary()

In [ ]:
model_elu.summary()

In [ ]:
# Relu

history_dict = history_relu.history

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_dict['loss'], label='Training Loss', color='blue')
plt.plot(history_dict['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_dict['mean_absolute_error'], label='Training MAE', color='blue')
plt.plot(history_dict['val_mean_absolute_error'], label='Validation MAE', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Sigmoid

history_dict2 = history_sigmoid.history

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_dict2['loss'], label='Training Loss', color='blue')
plt.plot(history_dict2['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_dict2['mean_absolute_error'], label='Training MAE', color='blue')
plt.plot(history_dict2['val_mean_absolute_error'], label='Validation MAE', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Tanh

history_dict3 = history_tanh.history

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_dict3['loss'], label='Training Loss', color='blue')
plt.plot(history_dict3['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_dict3['mean_absolute_error'], label='Training MAE', color='blue')
plt.plot(history_dict3['val_mean_absolute_error'], label='Validation MAE', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Elu

history_dict4 = history_elu.history

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_dict4['loss'], label='Training Loss', color='blue')
plt.plot(history_dict4['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_dict4['mean_absolute_error'], label='Training MAE', color='blue')
plt.plot(history_dict4['val_mean_absolute_error'], label='Validation MAE', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred_relu = model_relu.predict(x_test_scaled)
y_pred_sigmoid = model_sigmoid.predict(x_test_scaled)
y_pred_tanh = model_tanh.predict(x_test_scaled)
y_pred_elu = model_elu.predict(x_test_scaled)

# Calcular métricas para el modelo con ReLU
mse_relu = mean_squared_error(y_test, y_pred_relu)
mae_relu = mean_absolute_error(y_test, y_pred_relu)
r2_relu = r2_score(y_test, y_pred_relu)

print("\nMétricas para el modelo con ReLU:")
print(f"Mean Squared Error (MSE): {mse_relu:.2f}")
print(f"Mean Absolute Error (MAE): {mae_relu:.2f}")
print(f"R² Score: {r2_relu * 100:.2f}%")

# Calcular métricas para el modelo con Sigmoid
mse_sigmoid = mean_squared_error(y_test, y_pred_sigmoid)
mae_sigmoid = mean_absolute_error(y_test, y_pred_sigmoid)
r2_sigmoid = r2_score(y_test, y_pred_sigmoid)

print("\nMétricas para el modelo con Sigmoid:")
print(f"Mean Squared Error (MSE): {mse_sigmoid:.2f}")
print(f"Mean Absolute Error (MAE): {mae_sigmoid:.2f}")
print(f"R² Score: {r2_sigmoid * 100:.2f}%")

# Calcular métricas para el modelo con Tanh
mse_tanh = mean_squared_error(y_test, y_pred_tanh)
mae_tanh = mean_absolute_error(y_test, y_pred_tanh)
r2_tanh = r2_score(y_test, y_pred_tanh)

print("\nMétricas para el modelo con Sigmoid:")
print(f"Mean Squared Error (MSE): {mse_tanh:.2f}")
print(f"Mean Absolute Error (MAE): {mae_tanh:.2f}")
print(f"R² Score: {r2_tanh * 100:.2f}%")

# Calcular métricas para el modelo con Elu
mse_elu = mean_squared_error(y_test, y_pred_elu)
mae_elu = mean_absolute_error(y_test, y_pred_elu)
r2_elu = r2_score(y_test, y_pred_elu)

print("\nMétricas para el modelo con Sigmoid:")
print(f"Mean Squared Error (MSE): {mse_elu:.2f}")
print(f"Mean Absolute Error (MAE): {mae_elu:.2f}")
print(f"R² Score: {r2_elu * 100:.2f}%")

In [ ]:
plt.figure(figsize=(12, 10))

# Gráfico de dispersión para el modelo con ReLU
plt.subplot(2, 2, 1)
plt.scatter(y_test, y_pred_relu, alpha=0.5, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Valores Reales')
plt.ylabel('Valores Predichos')
plt.title('Modelo con ReLU')

# Gráfico de dispersión para el modelo con Sigmoid
plt.subplot(2, 2, 2)
plt.scatter(y_test, y_pred_sigmoid, alpha=0.5, color='orange')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Valores Reales')
plt.ylabel('Valores Predichos')
plt.title('Modelo con Sigmoid')

# Gráfico de dispersión para el modelo con Tanh
plt.subplot(2, 2, 3)
plt.scatter(y_test, y_pred_tanh, alpha=0.5, color='green')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Valores Reales')
plt.ylabel('Valores Predichos')
plt.title('Modelo con Tanh')

# Gráfico de dispersión para el modelo con Elu
plt.subplot(2, 2, 4)
plt.scatter(y_test, y_pred_elu, alpha=0.5, color='red')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Valores Reales')
plt.ylabel('Valores Predichos')
plt.title('Modelo con Elu')

plt.tight_layout()
plt.show()

In [ ]:

plt.figure(figsize=(12, 10))

residuals_relu = y_test - y_pred_relu.flatten()
plt.subplot(2, 2, 1)
plt.scatter(y_pred_relu, residuals_relu, alpha=0.5, color='blue')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Valores Predichos')
plt.ylabel('Residuales')
plt.title('Residuales para el Modelo con ReLU')

residuals_sigmoid = y_test - y_pred_sigmoid.flatten()
plt.subplot(2, 2, 2)
plt.scatter(y_pred_sigmoid, residuals_sigmoid, alpha=0.5, color='orange')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Valores Predichos')
plt.ylabel('Residuales')
plt.title('Residuales para el Modelo con Sigmoid')

residuals_tanh = y_test - y_pred_tanh.flatten()
plt.subplot(2, 2, 3)
plt.scatter(y_pred_tanh, residuals_tanh, alpha=0.5, color='green')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Valores Predichos')
plt.ylabel('Residuales')
plt.title('Residuales para el Modelo con Tanh')

residuals_elu = y_test - y_pred_elu.flatten()
plt.subplot(2, 2, 4)
plt.scatter(y_pred_elu, residuals_elu, alpha=0.5, color='green')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Valores Predichos')
plt.ylabel('Residuales')
plt.title('Residuales para el Modelo con Elu')

plt.tight_layout()
plt.show()

**1.1.4 Análisis de resultados**
___

### 1.2 Datos jugadores del FIFA
El dataset en la actividad incluye información sobre la performance de cada jugador en el juego FIFA. 

1. Tratamiento de los datos. Cada jugador tiene una posición determinada `team_position` y según la columna que tiene el mismo valor, tiene un puntaje. (0.5 pts)
   1. De acuerdo a ese puntaje, asignar las siguientes categorías:
      * Poor:[46.0, 62.0]
      * Interm:[63.0, 66.0]	
      * Good:[67.0, 71.0]	
      * Excel:[72.0, 94.0]   
2. Generación del modelo de clasificación, tomando en cuenta esta nueva columna a partir de las columnas (2.5 pts)
   * Considera las siguientes columnas como features: `attacking_crossing	attacking_finishing	attacking_heading_accuracy	attacking_short_passing	attacking_volleys	skill_dribbling	skill_curve	skill_fk_accuracy	skill_long_passing	skill_ball_control	movement_acceleration	movement_sprint_speed	movement_agility	movement_reactions	movement_balance	power_shot_power	power_jumping	power_stamina	power_strength	power_long_shots	mentality_aggression	mentality_interceptions	mentality_positioning	mentality_vision	mentality_penalties	mentality_composure	defending_marking	defending_standing_tackle	defending_sliding_tackle	goalkeeping_diving	goalkeeping_handling	goalkeeping_kicking	goalkeeping_positioning	goalkeeping_reflexes`. 
3. Generar el análisis y gráficas para analizar la performance del modelo. (2 pts)

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_players = pd.read_csv('players_20.csv')
data_players.head(3)

In [ ]:
data_players['team_position'].unique()

**1.2.1 Tratamiento de datos**
___

In [55]:
data_players['team_position'] = data_players['team_position'].str.lower()

In [56]:
# Borramos los que tengan NA 
columnas_a_verificar = ['rw', 'lw', 'cam', 'rcm', 'lcb', 'st', 'cdm', 'ldm', 'rm',
                        'rcb', 'lcm', 'lm', 'cf', 'lb', 'ls', 'rb', 'rdm',
                        'ram', 'rs', 'rf', 'cm', 'cb', 'lf', 'lam', 'rwb', 'lwb']

data_players = data_players.dropna(subset=columnas_a_verificar)

# Nos quedamos solo con los números antes del "+"
def extract_main_score(score):
    try:
        if pd.notna(score) and '+' in str(score):
            return int(str(score).split('+')[0])
        elif pd.notna(score):
            return int(score)
    except ValueError:
        return np.nan

for columna in columnas_a_verificar:
    data_players[columna] = data_players[columna].apply(extract_main_score)

In [ ]:
data_players.head(3)

In [58]:
data_players = data_players[data_players['team_position'].isin(columnas_a_verificar)]

def get_score_from_position(row):
    position = row['team_position']
    if position in data_players.columns and pd.notna(row[position]):
        return row[position]
    else:
        return None

data_players['score_from_position'] = data_players.apply(get_score_from_position, axis=1)

In [ ]:
data_players

In [60]:
def categorize_performance(score):
    if 46 <= score <= 62:
        return 'Poor'
    elif 63 <= score <= 66:
        return 'Interm'
    elif 67 <= score <= 71:
        return 'Good'
    elif 72 <= score <= 94:
        return 'Excel'
    else:
        return 'Unknown'

# Aplicamos la función para crear la nueva columna categorizada
data_players['categoria_segun_position'] = data_players['score_from_position'].apply(categorize_performance)

# Eliminamos las filas con la categoría 'Unknown' (si existieran)
data_players = data_players[data_players['categoria_segun_position'] != 'Unknown']

In [ ]:
data_players['categoria_segun_position'].value_counts()

**1.2.2 Modelo de clasificación**
___

In [62]:
features = [
    'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
    'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
    'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
    'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
    'movement_reactions', 'movement_balance', 'power_shot_power',
    'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
    'mentality_aggression', 'mentality_interceptions', 'mentality_positioning',
    'mentality_vision', 'mentality_penalties', 'mentality_composure',
    'defending_marking', 'defending_standing_tackle', 'defending_sliding_tackle',
    'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
    'goalkeeping_positioning', 'goalkeeping_reflexes'
]

x_players = data_players[features]
y_players = data_players['categoria_segun_position']

In [63]:
from tensorflow import keras
import tensorflow as tf

tf.random.set_seed(
    42
)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_players)

x_train_players, x_test_players, y_train_players, y_test_players = train_test_split(x_players, y_encoded, test_size=0.2, random_state=42)

input_dim = x_players.shape[1]

model_players = keras.models.Sequential()
model_players.add(keras.layers.Input(shape=(input_dim,)))

for _ in range(10):
    model_players.add(keras.layers.Dense(100, activation='relu'))
model_players.add(keras.layers.Dense(4, activation='softmax'))

In [64]:
model_players.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
history_cat_players = model_players.fit(
    x_train_players, 
    y_train_players, 
    epochs=100, 
    batch_size=32, 
    validation_data=(x_test_players, y_test_players)
)

In [ ]:
model_players.summary()

In [ ]:
from sklearn.metrics import classification_report

y_pred = model_players.predict(x_test_players)
y_pred_classes = y_pred.argmax(axis=1)

report = classification_report(y_test_players, y_pred_classes, target_names=label_encoder.classes_)
print(report)

In [ ]:
history_players = history_cat_players.history

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_players['loss'], label='Training Loss', color='blue')
plt.plot(history_players['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim(0,1)
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_players['accuracy'], label='Training Accuracy', color='blue')
plt.plot(history_players['val_accuracy'], label='Validation Accuracy', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim(0,1)
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
history_players = history_cat_players.history

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_players['loss'], label='Training Loss', color='blue')
plt.plot(history_players['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim(0,1)
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_players['accuracy'], label='Training Accuracy', color='blue')
plt.plot(history_players['val_accuracy'], label='Validation Accuracy', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim(0,1)
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test_players, y_pred_classes)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

**1.2.3 Análisis de graficos de la performance del modelo**
___

**1.2.4 Análisis de resultados**
___

## Parte 2: Índice latinoamericano de IA
Revisar el índide Latinoamericano de Inteligencia Artificial https://indicelatam.cl/. También disponible en PDF en el blackboard.
Responder a las siguientes preguntas:

1. ¿Cuál es el objetivo principal del Índice Latinoamericano de IA y cómo se mide el progreso en la región? **(0.5 pts)**


> El ILIA tiene como objetivo ayudar a que la Inteligencia Artificial se desarrolle de manera más inclusiva en América Latina. Busca aprovechar las oportunidades, identificar las diferencias que aún existen, y ayudar a impulsar acciones concretas para que la IA avance de manera beneficiosa para todos en la región. Para medir cómo vamos, el índice usa más de 70 indicadores, organizados en dimensiones como Factores Habilitantes, Investigación, Desarrollo y Gobernanza. 

2. ¿Qué países de Latinoamérica lideran actualmente el ranking del Índice Latinoamericano de IA y por qué? **(0.5 pts)**

> En el ILIA 2024, los países que están liderando el camino son Chile, Brasil y Uruguay. Lo que los hace destacar es su trabajo en diferentes áreas clave, como tener una buena infraestructura tecnológica, fomentar el talento especializado, promover la ciencia y la innovación. Estos países se encuentran en lo que se llama el "Cuadrante I", lo que básicamente significa que tienen un entorno bastante favorable para el desarrollo de la IA y, por eso, están bien posicionados para ser líderes regionales.

3. ¿Cuáles son los principales desafíos que enfrenta la región para avanzar en la adopción y desarrollo de la Inteligencia Artificial? **(0.5 pts)**

> Algunos de los grandes problemas que enfrenta la región para desarrollar la IA incluyen la falta de talento especializado y la dificultad para retener a los expertos que se forman, ya que muchos terminan yéndose a otros países. Además, falta infraestructura adecuada, como computadoras de alto rendimiento y acceso a redes rápidas. También hay una gran brecha de género, lo que significa que las mujeres aún están subrepresentadas tanto en los estudios como en el uso de la IA.

4. ¿De qué manera el Índice Latinoamericano de IA considera la ética y la responsabilidad social en el desarrollo de la IA? **(0.5 pts)**

> El índice también se preocupa por el aspecto ético y la responsabilidad social en el desarrollo de la IA. Evalúa qué tan bien los países están integrando estrategias de IA que respeten estos valores, incluyendo marcos regulatorios para asegurarse de que todo se haga de manera responsable. Por ejemplo, menciona la Metodología RAM de la UNESCO, que sirve como una guía para lograr una IA más ética. Además, hay iniciativas conjuntas con universidades para crear algoritmos más transparentes y justos.

5. ¿Qué iniciativas gubernamentales y privadas están impulsando el desarrollo de la IA en Latinoamérica y cómo se reflejan en el Índice Latinoamericano de IA? **(0.5 pts)**

> En América Latina, hay iniciativas tanto del gobierno como del sector privado para apoyar el desarrollo de la IA. Algunos ejemplos incluyen la inversión en infraestructura tecnológica con apoyo de la CAF y la creación del Grupo de Trabajo por la Ética de la IA, que nació durante la Cumbre de Santiago. También hay programas del BID que buscan ayudar a que más empresas adopten la IA. A nivel privado, grandes empresas como Google, Microsoft y AWS están poniendo su grano de arena para que el desarrollo de la IA en la región sea más equilibrado y responsable.

6. De acuerdo al índice latinoamericano de IA ¿Cuál es el potencial del desarrollo de la inteligencia artificial en el Perú? **(0.5 pts)**

> Perú tiene un gran potencial, pero enfrenta algunos desafíos importantes. En el índice, Perú está en el "Cuadrante III" en lo que respecta a infraestructura y desarrollo de la IA, lo que significa que hay mucho por mejorar. Sin embargo, en el aspecto de gobernanza, está un poco mejor, en el "Cuadrante II", lo que indica que hay un buen ambiente regulatorio. Si Perú logra mejorar en las áreas donde hay más problemas, podría aprovechar mucho mejor las oportunidades que trae la IA.

7. ¿Qué iniciativas puede tomar en el Perú para ascender en el índice latinoamericano? **(1 pts)**

> Para que Perú pueda avanzar en el Índice Latinoamericano de Inteligencia Artificial, lo primero es ponerle atención a la infraestructura tecnológica: mejorar la conectividad, apostar por el 5G y contar con mejor capacidad de computación. Sin esto, no se puede competir. Además, hay que trabajar en formar y retener talento especializado. No basta con educar, también hay que hacer atractivo quedarse en el país. Es clave tener acceso a datos de calidad, y para eso se necesita infraestructura que los respalde y políticas que aseguren su uso responsable. También necesitamos marcos regulatorios que promuevan la IA de forma ética, apoyando tanto la inversión pública como la privada.

## Parte 3: Historia de la Inteligencia Artificial

**Objetivo:** Extraer 4 oraciones clave y 3 ideas (del grupo) sobre el capítulo.

1. Capítulo 1
   1. **"Turing was, for all practical purposes, the inventor of the computer, and shortly after that, he largely invented the field of AI."**
   - "Turing fue, a todos los efectos prácticos, el inventor de la computadora, y poco después de eso, inventó en gran medida el campo de la IA"
   2. **"And if all mathematical decision problems can be solved by following a recipe, then for any decision problem, you should be able to design a Turing machine to solve it."**
   - "Y si todos los problemas de decisión matemática se pueden resolver siguiendo una receta, entonces para cualquier problema de decisión, debería ser posible diseñar una máquina de Turing para resolverlo"
   3. **"The question of whether AI is possible ultimately amounts to whether we can produce intelligent behavior simply by following lists of instructions like these."**
   - La cuestión de si la IA es posible, en última instancia, se reduce a si podemos producir un comportamiento inteligente simplemente siguiendo listas de instrucciones como estas"
   4. **"Although computers are just machines for following instructions, this does not mean that they are incapable of making decisions."**
   - "Aunque las computadoras son simplemente máquinas para seguir instrucciones, esto no significa que sean incapaces de tomar decisiones".

1. Capítulo 2
   1. **"The next two decades were the first boom in AI. There was a flush of optimism, growth, and apparent progress, leading to the era called the golden age of AI, from about 1956 to 1974."**
   - "Las siguientes dos décadas fueron el primer auge de la IA. Hubo un auge de optimismo, crecimiento y aparente progreso, que condujo a la era llamada la edad de oro de la IA, desde aproximadamente 1956 hasta 1974."
   2. **"Instead of starting out trying to build a complete general intelligent system, the approach adopted was to identify the various different individual capabilities that seemed to be required for general-purpose AI and to build systems that could demonstrate these capabilities."**
   - "En lugar de comenzar a intentar construir un sistema inteligente general completo, el enfoque adoptado fue identificar las distintas capacidades individuales que parecían necesarias para una IA de propósito general y construir sistemas que pudieran demostrar esas capacidades."
   3. **"There was a good deal of naivety in the golden age, with researchers making reckless and grandiose predictions about the likely speed of progress in the field, which have haunted AI ever since."**
   - "Hubo mucha ingenuidad en la edad de oro, con investigadores haciendo predicciones imprudentes y grandiosas sobre la posible velocidad de progreso en el campo, lo que ha perseguido a la IA desde entonces."
   4. **"The decade from the early 1970s to the early 1980s later became known as the AI winter, although it should perhaps better be known as the first AI winter, because there were more to come"**
   - "La década que va desde principios de los años 1970 hasta principios de los años 1980 se conoció más tarde como el invierno de la IA, aunque tal vez debería llamarse mejor el primer invierno de la IA, porque aún quedaban más por venir."

1. Capítulo 6
   1. **" In the second decade of the twenty-first century, AI has attracted more interest than any new technology since the World Wide Web in the 1990s."**
   - "En la segunda década del siglo XXI, la IA ha atraído más interés que cualquier otra tecnología nueva desde la aparición de la World Wide Web en los años 90."
   2. **" Everywhere that technology is used, AI is finding applications: in education, science, industry, commerce, agriculture, health care, entertainment, the media and arts, and beyond."**
   - "En todos los lugares donde se utiliza la tecnología, la IA está encontrando aplicaciones: en la educación, la ciencia, la industria, el comercio, la agricultura, la atención sanitaria, el entretenimiento, los medios de comunicación y las artes, y más allá."
   3. **"One important new opportunity for AI-powered health care is what we might call personal health care management."**
   - "Una nueva e importante oportunidad para la atención sanitaria basada en inteligencia artificial es lo que podríamos llamar la gestión de la atención sanitaria personal."
   4. **"If you could find a way for a car to know precisely where it was and what was around it, then you would have solved the problem of driverless cars. The solution to this problem was to be modern machine learning techniques: without them, driverless cars would not be possible"**
   - "Si se pudiera encontrar una manera de que un coche supiera exactamente dónde se encuentra y qué hay a su alrededor, se habría solucionado el problema de los coches sin conductor. La solución a este problema eran las técnicas modernas de aprendizaje automático: sin ellas, los coches sin conductor no serían posibles."

1. Ideas (del grupo)
   1. Alan Turing, con su tería de las máquinas que siguen instrucciones (Máquinas de Turing), creó las bases de las computadoras actuales y de la IA. Los medios introducieron el término "cerebros electrónicos" para describir los primeros ordenadores, lo que dio la impresión errónea de que estas máquinas poseían algún tipo de inteligencia superior. Sin embargo, los ordenadores no eran inteligentes; solo seguían instrucciones, llamadas algoritmos.
   2. En la época dorada de la inteligencia artifial se realizaron muchos avances y se tenia muchas espectativas muy poco realistas de lo que sería capaz de realizar esta tecnologia en el contexto de la época. Sin embargo, pronto se encotraron con barreras debido a las limitaciones tecnologias y debido a criticas fue perdiendo la inversión con la que contaba inicialmente.
   3. El deep learning ha permitido que la inteligencia artificial encuentre aplicaciones en casi todos los aspectos de la vida, desde la medicina hasta la creación de imágenes y la conducción autónoma.